In [ ]:
import cvxpy as cp
from cvxpylayers.torch import CvxpyLayer
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import torch

from dimp.robots import (
    PointMassInput, PointMassRobot, PointMassState, MPCData
)

from dimp.utils import init_matplotlib, get_colors

init_matplotlib()

### Create the Data for the MPC

In [ ]:
nr = 2

ns = PointMassState.n   # number of states (x, y, vx, vy)
ni = PointMassInput.n   # number of inputs (ax, ay)

nc = 10     # number of control intervals

# Note that class MPCData does initialize all the cp variables and parameters.
robots_data = [MPCData(
    nc=nc,
    states_list=[PointMassState(cp.Variable(ns, name=f"s{i}_{k+1}")) for k in range(nc)],
    statesbar_list=[PointMassState(cp.Parameter(ns)) for _ in range(nc)],
    inputs_list=[PointMassInput(cp.Variable(ni, name=f"u{i}_{k}")) for k in range(nc)],
    inputsbar_list=[PointMassInput(cp.Parameter(ni)) for _ in range(nc)],
) for i in range(nr)]

dt = 0.1    # sampling time

rob_mass = [1.0 for _ in range(nr)]     # robot masses

# Parameters
obj_size = 1.0
obj_mass = 1.0
friction_coeff = 0.5

In [ ]:
nso = 4     # number of object states (x, y, vx, vy)

so_0 = cp.Parameter(nso, name="so_0")     # object initial state
so_data = MPCData(
    nc=nc,
    states_list=[PointMassState(so_0)] + [PointMassState(cp.Variable(nso, name=f"so_{k+1}")) for k in range(nc)],
    statesbar_list=[PointMassState(so_0)] + [PointMassState(cp.Parameter(nso)) for _ in range(nc)],
    inputs_list=[],
    inputsbar_list=[],
)

forces_data = [MPCData(
    nc=nc,
    states_list=[],
    statesbar_list=[],
    inputs_list=[PointMassInput(cp.Variable(ni, name=f"f{i}_{k}")) for k in range(nc)],
    inputsbar_list=[PointMassInput(cp.Parameter(ni)) for _ in range(nc)],
) for i in range(nr)]

goal_param = cp.Parameter(2, name="goal")  # (x_goal, y_goal)

### Create the MPC Problem

In [ ]:
def generate_constraints():
    # p_k+1 - (p_k + v_k*dt) == 0
    # v_k+1 - (v_k + u_k*dt + fc_k*dt) == 0 
    rob_dyn_constraints = [[
        cp.hstack([
            robots_data[i].xi[k+1] - (robots_data[i].xi[k] + robots_data[i].vxi[k]*dt),
            robots_data[i].yi[k+1] - (robots_data[i].yi[k] + robots_data[i].vyi[k]*dt),
            robots_data[i].vxi[k+1] - (robots_data[i].vxi[k] + (robots_data[i].fxi[k] - forces_data[i].fxi[k]) / rob_mass[i] * dt),
            robots_data[i].vyi[k+1] - (robots_data[i].vyi[k] + (robots_data[i].fyi[k] - forces_data[i].fyi[k]) / rob_mass[i] * dt),
        ]) == 0
        for k in range(nc)
    ] for i in range(nr)]

    # p_k+1 - (p_k + v_k*dt) == 0
    # v_k+1 - (v_k + sum(forces_i_k)/m*dt) == 0
    obj_dyn_constraints = [
        cp.hstack([
            so_data.xi[k+1] - (so_data.xi[k] + so_data.vxi[k]*dt),
            so_data.yi[k+1] - (so_data.yi[k] + so_data.vyi[k]*dt),
            so_data.vxi[k+1] - (so_data.vxi[k] + cp.sum([forces_data[i].fxi[k] for i in range(nr)]) / obj_mass * dt),
            so_data.vyi[k+1] - (so_data.vyi[k] + cp.sum([forces_data[i].fyi[k] for i in range(nr)]) / obj_mass * dt),
        ]) == 0
        for k in range(nc)
    ]

    input_limits_constraints = [[
        cp.norm(robots_data[i].inputi[k], p=2) <= 10.0
        for k in range(nc)
    ] for i in range(nr)]

    # normals[i = 0..nr-1][k = 0..nc-1]
    normals = [[
        (cp.hstack([
            (robots_data[i].xi[k] - so_data.xi[k]),
            (robots_data[i].yi[k] - so_data.yi[k]),
        ]) / (cp.norm(cp.hstack([
            (robots_data[i].xi[k] - so_data.xi[k]),
            (robots_data[i].yi[k] - so_data.yi[k]),
        ]), 2) + 1e-6)).value
        for k in range(nc)
    ] for i in range(nr)]
    
    for n_l in normals:
        for n in n_l:
            assert not np.any(np.isnan(n)), "NaN in normal vector!"
            assert not np.any(np.isinf(n)), "Inf in normal vector!"

    # tangents[i = 0..nr-1][k = 0..nc-1]
    tangents = [[
        cp.hstack([
            -normals[i][k][1],
            normals[i][k][0],
        ])
        for k in range(nc)
    ] for i in range(nr)]

    s_no_interp = cp.Variable(nr*nc, name="s_no_interp")
    #! Nonconvex!
    # |n^T (p_rob - p_obj)| >= obj_size/2 + s_no_interp
    no_interp_constr = [[
        cp.abs(cp.transpose(normals[i][k]) @ cp.hstack([
            robots_data[i].xi[k] - so_data.xi[k],
            robots_data[i].yi[k] - so_data.yi[k],
        ])) - obj_size/2 >= s_no_interp[i*nc + k]
        for k in range(nc)
    ] for i in range(nr)]

    normal_force_constraints = [[
        cp.transpose(normals[i][k]) @ cp.hstack([
            forces_data[i].fxi[k],
            forces_data[i].fyi[k],
        ]) >= 0
        for k in range(nc)
    ] for i in range(nr)]

    s_friction = cp.Variable(nr*nc, name="s_friction")
    # |tangents[i] . f_i| <= mu * (normals[i] . f_i)
    friction_constr = [[
        cp.abs(cp.transpose(tangents[i][k]) @ cp.hstack([
            forces_data[i].fxi[k],
            forces_data[i].fyi[k],
        ])) <= friction_coeff * (cp.transpose(normals[i][k]) @ cp.hstack([
            forces_data[i].fxi[k],
            forces_data[i].fyi[k],
        ])) + s_friction[i*nc + k]
        for k in range(nc)
    ] for i in range(nr)]

    # tangents[i] . (v_obj - v_rob) == 0
    sticking_kinematics_constr = [[
        cp.transpose(tangents[i][k]) @ cp.hstack([
            so_data.vxi[k] - robots_data[i].vxi[k],
            so_data.vyi[k] - robots_data[i].vyi[k],
        ]) == 0
        for k in range(nc)
    ] for i in range(nr)]
    
    constraints = sum([
        [s_no_interp >= 0],
        [s_friction >= 0],
        sum(rob_dyn_constraints, []),
        obj_dyn_constraints,
        sum(input_limits_constraints, []),
        sum(normal_force_constraints, []),
        sum(friction_constr, []),
        sum(no_interp_constr, []),
        sum(sticking_kinematics_constr, []),
    ], [])
    
    slacks = [
        s_no_interp,
        s_friction,
    ]
    
    return constraints, slacks

In [ ]:
# plot_res is only for plotting the results. Ignore.
def plot_res(robots_data=robots_data, so_data=so_data, forces_data=forces_data):
    times = np.arange(nc) * dt
    
    fig, axs = plt.subplots(3, 2, figsize=(9.6, 7.2))
    
    axs[0, 0].plot(
        np.arange(nc+1) * dt,
        [so_data.xi[i].value for i in range(nc+1)],
        label='Object',
    )
    for i in range(nr):
        axs[0, 0].plot(
            np.arange(nc+1) * dt,
            [robots_data[i].xi[k].value for k in range(nc+1)],
            label=f'Robot {i}',
        )
        axs[0, 1].plot(
            np.arange(nc+1) * dt,
            [robots_data[i].yi[k].value for k in range(nc+1)],
            label=f'Robot {i}',
        )
    axs[0, 0].set_xlabel('time')
    axs[0, 0].set_ylabel('x')
    axs[0, 0].set_title('x coordinate')
    axs[0, 0].legend()
    
    axs[0, 1].set_xlabel('time')
    axs[0, 1].set_ylabel('y')
    axs[0, 1].set_title('y coordinate')
    axs[0, 1].legend()
    
    axs[1, 0].plot(
        [so_data.xi[i].value for i in range(nc+1)],
        [so_data.yi[i].value for i in range(nc+1)],
        label='Object',
    )
    for i in range(nr):
        axs[1, 0].plot(
            [robots_data[i].xi[k].value for k in range(nc+1)],
            [robots_data[i].yi[k].value for k in range(nc+1)],
            label=f'Robot {i}',
        )
    axs[1, 0].set_xlabel('x')
    axs[1, 0].set_ylabel('y')
    axs[1, 0].set_title('Trajectories')
    axs[1, 0].legend()
    
    for i in range(nr):
        axs[2, 0].plot(
            times,
            [robots_data[i].fxi[k].value for k in range(nc)],
            label=f'Robot {i} $f_x$',
        )
        axs[2, 1].plot(
            times,
            [robots_data[i].fyi[k].value for k in range(nc)],
            label=f'Robot {i} $f_y$',
        )
    axs[2, 0].set_xlabel('Time step')
    axs[2, 0].set_ylabel('Force x')
    axs[2, 0].set_title('Forces x')
    axs[2, 0].legend()
    axs[2, 1].set_xlabel('Time step')
    axs[2, 1].set_ylabel('Force y')
    axs[2, 1].set_title('Forces y')
    axs[2, 1].legend()

In [ ]:
# plot_anim is only for plotting the results. Ignore.
def plot_anim():
    xm = np.min([
        np.array([robots_data[0].xi[k].value for k in range(nc+1)]).min(),
        np.array([robots_data[1].xi[k].value for k in range(nc+1)]).min(),
        np.array([so_data.xi[k].value for k in range(nc+1)]).min()
    ]) - 1.0
    xM = np.max([
        np.array([robots_data[0].xi[k].value for k in range(nc+1)]).max(),
        np.array([robots_data[1].xi[k].value for k in range(nc+1)]).max(),
        np.array([so_data.xi[k].value for k in range(nc+1)]).max()
    ]) + 1.0
    ym = np.min([
        np.array([robots_data[0].yi[k].value for k in range(nc+1)]).min(),
        np.array([robots_data[1].yi[k].value for k in range(nc+1)]).min(),
        np.array([so_data.yi[k].value for k in range(nc+1)]).min()
    ]) - 1.0
    yM = np.max([
        np.array([robots_data[0].yi[k].value for k in range(nc+1)]).max(),
        np.array([robots_data[1].yi[k].value for k in range(nc+1)]).max(),
        np.array([so_data.yi[k].value for k in range(nc+1)]).max()
    ]) + 1.0
    
    colors = get_colors()
    
    fig = go.Figure(
        data=[go.Scatter(
            x=[robots_data[i].xi[0].value[0]],
            y=[robots_data[i].yi[0].value[0]],
            mode='markers',
            marker=dict(size=10, color=colors[i]),
            name=f'Robot {i}'
        ) for i in range(nr)] + [
            go.Scatter(
                x=[so_data.xi[0].value[0]],
                y=[so_data.yi[0].value[0]],
                mode='markers',
                marker=dict(size=10, color='black'),
                name='Object'
        )]
    )
    
    fig.update_layout(
        xaxis=dict(range=[xm, xM], autorange=False),
        yaxis=dict(range=[ym, yM], autorange=False),
        title="Point Mass Pushing Animation",
        updatemenus=[dict(
            type="buttons",
            buttons=[dict(label="Play",
                          method="animate",
                          args=[None, {"frame": {"duration": 500, "redraw": True},
                                       "fromcurrent": True, "transition": {"duration": 0}}]),
                     dict(label="Pause",
                          method="animate",
                          args=[[None], {"frame": {"duration": 0, "redraw": False},
                                         "mode": "immediate",
                                         "transition": {"duration": 0}}])]
        )]
    )
    
    fig.update(
        frames=[go.Frame(
            data=[go.Scatter(
                x=[robots_data[i].xi[k].value[0]],
                y=[robots_data[i].yi[k].value[0]],
                mode='markers',
                marker=dict(size=10, color=colors[i]),
                name=f'Robot {i}'
            ) for i in range(nr)] + [go.Scatter(
                x=[so_data.xi[k].value[0]],
                y=[so_data.yi[k].value[0]],
                mode='markers',
                marker=dict(size=10, color='black'),
                name='Object'
            )]
        ) for k in range(1, nc+1)]
    )
    
    fig.show()

### Optimization Problem Formulation

$$
\begin{align}
& \min_{x} && \sum_{k=1}^{n_c} \sum_{i=i}^{n_r} \left\lVert s_{ik} - s_{ik}^{ref} \right\rVert_{Q}^2 + \left\lVert u_{ik} \right\rVert_{R}^2 \\
& \text{s.t.} && p_{ik+1} = p{ik} + v_{ik} \Delta t && \text{robot dyn} \\
&&& v_{ik+1} = v_{ik} + u_{ik} \Delta t - \frac{f_{ik}}{m_i} \Delta t && \text{robot dyn} \\
&&& p_{ok+1} = p_{ok} + v_{ik} \Delta t && \text{object dyn} \\
&&& v_{ok+1} = v_{ok} + \sum_{i=1}^{n_r} \frac{f_{ik}}{m_o} \Delta t && \text{object dyn} \\
&&& u_{ik} \in \mathcal{U}_{i} && \text{input limits} \\
&&& \overline{n}_{ik}^T (p_{ik} - p_{ok}) - r_o \leq s_{interp} && \text{no interpenetration} \\
&&& \overline{n}_{ik}^T f_{ik} \geq 0 && \text{contact force dir} \\
&&& \overline{t}_{ik}^T f_{ik} \leq \mu_i \overline{n}_{ik}^T f_{ik} && \text{friction cone} \\
&&& \overline{t}_{ik}^T (v_{ok} - v_{ik}) = 0 && \text{stick contact}
\end{align}
$$
where:
- $\overline{n}_{ik}$: contact (with object) normal vector at time step k for robot i. Outward from object.
- $\overline{t}_{ik}$: contact (with object) tangential vector at time step k for robot i.

In [ ]:
def create_qp(constraints, slacks):
    objective = cp.Minimize(
        cp.sum_squares(so_data.xi[nc] - goal_param[0]) \
        + cp.sum_squares(so_data.yi[nc] - goal_param[1]) \
        + 10000 * cp.norm(slacks[0], p=2) \
        + 10000 * cp.norm(slacks[1], p=2)
    )
    
    problem = cp.Problem(objective, constraints)
    
    return problem

so_0.value = np.array([-obj_size/2, 0.0, 0.0, 0.0])
for elem in s0:
    elem.value = so_0.value
goal_param.value = np.array([5.0, 0.0])

constraints, slacks = generate_constraints()
problem = create_qp(constraints, slacks)
assert problem.is_dpp()

problem.solve(solver=cp.CLARABEL, verbose=False)

plot_anim()
plot_res(robots_data=robots_data, so_data=so_data, forces_data=forces_data)